In [1]:
import pandas as pd
import numpy as np
from faker import Faker
from collections import defaultdict
from sqlalchemy import create_engine
import datetime
import random
import re


In [2]:
from numpy import integer

Faker.seed(300)
random.seed(300)
fake=Faker()

Account= defaultdict(list)

row_number=50

In [3]:

for i in range (row_number):

    if i > (row_number-4):

        Account["İBAN"].append(fake.bothify(text='EU##############'))
    else :
        Account["İBAN"].append(fake.bothify(text='TR##############'))
    
    Account["ID"].append(fake.bothify(text='######'))
    
    
   
    random_size = random.randint(1,10)



In [4]:
len(Account["ID"])

50

In [5]:
Bank_Account=pd.DataFrame(Account)
Bank_Account

,İBAN,ID
0,TR95507586992216,559231
1,TR98801448184950,119364
2,TR09207709436363,428576
3,TR35962104639049,518915
4,TR02358285996177,860432
5,TR70813201077458,676589
6,TR64823755428349,103253
7,TR49594359355194,464853
8,TR33319101216019,183454
9,TR61501655751613,513223


In [6]:
Money_Transfer= defaultdict(list)

Money_Transfer_Size=300

In [7]:
for i in range (Money_Transfer_Size): 
   max=random.randint(3,6)
   min=random.randint(1,3)
   random_size=random.randint(min,max)
   if len(Money_Transfer["From_Bank_Id"])==Money_Transfer_Size:
               break
   for j in range (random_size):

         random_ıd_choice_from=random.choice(Account["ID"])
         random_ıd_choice_to=random.choice(Account["ID"])
         
         if random_ıd_choice_from==random_ıd_choice_to:
            i=i-1
            break
         Money_Transfer["From_Bank_Id"].append(random_ıd_choice_from)
         Money_Transfer["To_Bank_Id"].append(random_ıd_choice_to)
         
         if random_size <= 2:
            Money_Transfer["AMAOUNT"].append(fake.pyfloat(right_digits=2,positive=True,min_value=100000.00,max_value=250000.00))
         elif (random_size > 2 ) and (random_size <= 4):
            Money_Transfer["AMAOUNT"].append(fake.pyfloat(right_digits=2,positive=True,min_value=10000.00,max_value=100000.00))
         elif (random_size > 4) and (random_size <= 7)  :
            Money_Transfer["AMAOUNT"].append(fake.pyfloat(right_digits=2,positive=True,min_value=1000.00,max_value=10000.00))
         elif random_size  > 7  :
            Money_Transfer["AMAOUNT"].append(fake.pyfloat(right_digits=2,positive=True,min_value=5.00,max_value=1000.00))
         Money_Transfer["TimeStamp"].append(fake.date_time_ad(start_datetime=datetime.datetime(2022,9,14)))
         if len(Money_Transfer["From_Bank_Id"])==Money_Transfer_Size:
               break
   if len(Money_Transfer["From_Bank_Id"])==Money_Transfer_Size:
               break

In [8]:
len(Money_Transfer["To_Bank_Id"])

300

In [9]:
Money_Transfe_dF=pd.DataFrame(Money_Transfer)
Money_Transfe_dF

,From_Bank_Id,To_Bank_Id,AMAOUNT,TimeStamp
0,156816,692689,9050.89,2022-09-21 23:02:42
1,185775,513223,9865.96,2022-09-14 12:20:37
2,681895,338301,5673.14,2022-09-18 14:55:13
3,108309,995794,5622.22,2022-10-11 15:50:35
4,040466,923026,4167.38,2022-09-22 22:14:44
...,...,...,...,...
295,525506,788297,96028.98,2022-10-02 09:42:33
296,065464,172091,12500.97,2022-09-28 20:58:37
297,553875,692689,54014.32,2022-10-29 02:46:47
298,040595,553875,11399.81,2022-10-04 12:49:51


In [13]:
from neo4j import GraphDatabase

In [11]:
bank_transaction=Money_Transfe_dF.values.tolist()

In [14]:
uri = "neo4j://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "test"))

#Deleting existing values and relationships
def clear_db(tx):
    tx.run("MATCH(n) Detach Delete n")
#Creating customer node
def create_customer(tx, customer_id):
    tx.run("CREATE (c1:customer {name: $name, customer_id: $id})",name='customer', id=customer_id)

#Creating relationship between customer nodes
def create_transaction(tx,customer_id,other_customer_id,amount,timestamp):
    tx.run("MATCH (c1:customer),(c2:customer) "
    "Where c1.customer_id= $id and c2.customer_id= $otherid "
    "Create (c1)-[t:Transaction{amount: $amount, time: $timestamp}]->(c2)",
    id=customer_id,otherid=other_customer_id,amount=amount,timestamp=timestamp)


with driver.session() as session:

    session.execute_write(clear_db)

    session.execute_write()

driver.close()

Unable to retrieve routing information
Transaction failed and will be retried in 0.9299036297421857s (Unable to retrieve routing information)
Unable to retrieve routing information
Transaction failed and will be retried in 1.802615765007051s (Unable to retrieve routing information)


KeyboardInterrupt: 